In [96]:
import requests # to get the HTML
from bs4 import BeautifulSoup # to get parse the text
from selenium.webdriver.common.by import By
from selenium import webdriver
import chromedriver_binary
import numpy as np
import pandas as pd
import tqdm

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
url = 'https://www.jobs.bg/front_job_search.php?frompage=0&add_sh=1&from_hp=1&term=#paging'
#      https://www.jobs.bg/front_job_search.php?frompage=15&add_sh=1&from_hp=1&term=#paging

r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [93]:
def get_pages(soup):
    """Get the total number of job offerings and the total number of pages.
    output: total number of pages
    """
    total_job_number = soup.find('td',
                            attrs={'style':
                                  'height:25px;width:220px;font-weight:500;padding-bottom:5px;'}).text.split()[-1]
    total_job_number= int(total_job_number)
    total_pages = int(total_job_number/15)+1 #round up
    return total_pages

In [103]:
# total_pages = get_pages(soup)
total_pages = 30
results =[]

for page in tqdm.tqdm(range(total_pages)):

    url = f'https://www.jobs.bg/front_job_search.php?frompage={page*15}&add_sh=1&from_hp=1&term=#paging'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    jobs_list = soup.find('table',
                               attrs={'style':
                                       'width:740px;'}).find_all('tr')

    for job in jobs_list:
        date = job.find('span',attrs={'class':'card__subtitle mdc-typography mdc-typography--overline line-height-1-5'}).text.strip()
        rating = job.find('span',attrs={'class':'iconed'}).text.strip()
        position = job.find('a',attrs={'class':'card__title mdc-typography mdc-typography--headline6 text-overflow'}).text.strip()
        benefits = job.find('span',attrs={'class':'card__subtitle mdc-typography mdc-typography--body2 top-margin'}).text.strip()
        results.append([date, rating, position, benefits])

100%|██████████| 30/30 [00:40<00:00,  1.36s/it]


In [104]:
pd.DataFrame(results)

,0,1,2,3
0,днес,,Customer Service Advisor,София; Заплата от 1650 до 2150 BGN (Нето)
1,днес,,RETENTION SPECIALIST,София; Заплата от 5000 до 8000 BGN (Бруто)
2,днес,,Customer Care Agent with German,София; Възможност за работа от вкъщи; Дистанци...
3,днес,,"Purchasing Administrator with German, Italian,...",София; Възможност за работа от вкъщи; Дистанци...
4,днес,,Trading Platform Administrator,София; Дистанционно интервю
...,...,...,...,...
445,26.04.21,,"Global Process Owner, Order to Cash",София
446,26.04.21,,Международен шофьор / Международни шофьори,София; Заплата от 2400 до 2900 EUR (Нето)
447,26.04.21,,HR Administrator Interim,София; Дистанционно интервю
448,26.04.21,,Sr. Project Manager (User Enablement - Celonis),София; Възможност за работа от вкъщи; Дистанци...
